In [1]:
import ee
import geopandas as gpd
import geemap
import xarray as xr
import wxee

In [2]:
ee.Authenticate()
ee.Initialize(project='ee-nunezrimedio-tesina',opt_url='https://earthengine-highvolume.googleapis.com')

In [3]:
import ee
import geemap

# Initialize Earth Engine
ee.Initialize()

def get_province_geometry(province_name):
    provinces = ee.FeatureCollection("FAO/GAUL/2015/level1")
    return provinces.filter(ee.Filter.And(
        ee.Filter.eq('ADM0_NAME', 'Argentina'),
        ee.Filter.eq('ADM1_NAME', province_name)
    )).first().geometry()

# Get geometries for Santiago del Estero and Chaco
santiago_geometry = get_province_geometry('Santiago Del Estero')
chaco_geometry = get_province_geometry('Chaco')

# Find an area on the border
#border_area = santiago_geometry.intersection(chaco_geometry).centroid().buffer(3000).bounds()

border_area = ee.Geometry.Polygon([
    [
        [-61.831221, -25.658720],    # Northwest corner
        [-61.831221, -25.778083],    # Southwest corner
        [-61.711858, -25.778083],    # Southeast corner
        [-61.711858, -25.658720]     # Northeast corner
    ]
])


parana_delta = ee.Geometry.Polygon(
    [[[-59.5, -33.5],
      [-59.5, -34.0],
      [-58.8, -34.0],
      [-58.8, -33.5]]])

# Create a map centered on Argentina
Map = geemap.Map(center=[-27, -62], zoom=6)

# Add layers to the map
Map.addLayer(border_area, {'color': 'FF0000', 'fillColor': 'FF000088'}, 'Border Area')
Map.addLayer(parana_delta, {'color': '0000FF', 'fillColor': '0000FF88'}, 'Paraná Delta')

# Display the map
Map

Map(center=[-27, -62], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(ch…

In [6]:
dw_filtered_chaco = wxee.TimeSeries("GOOGLE/DYNAMICWORLD/V1").filterDate("2020-01-01", "2021-01-01").filterBounds(border_area).select("label")
dw_filtered_parana = wxee.TimeSeries("GOOGLE/DYNAMICWORLD/V1").filterDate("2020-01-01", "2021-01-01").filterBounds(parana_delta).select("label")

# Count available images in each region
chaco_count = dw_filtered_chaco.filterBounds(border_area).size().getInfo()
parana_count = dw_filtered_parana.filterBounds(parana_delta).size().getInfo()

print(f"Chaco images: {chaco_count}")
print(f"Paraná images: {parana_count}")

Chaco images: 50
Paraná images: 250


## Descarga Basica: Moda temporal de la banda label

In [20]:
# Descarga Paraná
aoi = parana_delta
dw_filtered = wxee.TimeSeries("GOOGLE/DYNAMICWORLD/V1").filterDate("2020-01-01", "2021-01-01").filterBounds(aoi).select("label")

def clip_image(image):
    return image.clip(aoi)

dw_filtered = dw_filtered.map(clip_image)

monthly_ts = dw_filtered.aggregate_time("year", ee.Reducer.mode())

ds = monthly_ts.wx.to_xarray(region=ee.Feature(aoi).geometry(), scale=30)
ds = ds.fillna(-1)
ds = ds.clip(min=-128,max=127)
ds["label"] = ds["label"].astype("int8")
ds["x"] = ds["x"].astype("float32")
ds["y"] = ds["y"].astype("float32")
ds["time"] = ds["time"].dt.strftime("%Y-%m")
ds.to_netcdf(rf"Data/DW/2020_parana_delta_mode.nc")

Requesting data:   0%|          | 0/1 [00:00<?, ?it/s]

In [7]:
# Descarga Chaco-Santiago
aoi = border_area
dw_filtered = wxee.TimeSeries("GOOGLE/DYNAMICWORLD/V1").filterDate("2020-01-01", "2021-01-01").filterBounds(aoi).select("label")

def clip_image(image):
    return image.clip(aoi)

dw_filtered = dw_filtered.map(clip_image)

monthly_ts = dw_filtered.aggregate_time("year", ee.Reducer.mode())

ds = monthly_ts.wx.to_xarray(region=ee.Feature(aoi).geometry(), scale=30)
ds = ds.fillna(-1)
ds = ds.clip(min=-128,max=127)
ds["label"] = ds["label"].astype("int8")
ds["x"] = ds["x"].astype("float32")
ds["y"] = ds["y"].astype("float32")
ds["time"] = ds["time"].dt.strftime("%Y-%m")
ds.to_netcdf(rf"Data/DW/2020_border_area_mode.nc")

Requesting data:   0%|          | 0/1 [00:00<?, ?it/s]

## Pooling Espacial de la banda label y agregación temporal con moda

In [22]:
## Paraná
aoi = parana_delta

# Load the Dynamic World dataset and filter by AOI and date range
dw_filtered = wxee.TimeSeries("GOOGLE/DYNAMICWORLD/V1") \
                .filterDate("2020-01-01", "2021-01-01") \
                .filterBounds(aoi) \
                .select("label")

# Clip images to the area of interest
def clip_image(image):
    return image.clip(aoi)

# Apply the clipping function to each image in the collection
dw_filtered = dw_filtered.map(clip_image)

# Aggregate temporally to get the mode for each year
yearly_ts = dw_filtered.aggregate_time("year", ee.Reducer.mode())

# Function to spatially pool the data using a 30m resolution
def spatial_pool(image):
    # Set the default projection for the image
    image = image.setDefaultProjection('EPSG:4326', None, 30)  # Set the projection with a scale of 30m

    return image.reduceResolution(
        reducer=ee.Reducer.mode(),
        maxPixels=65535,  # Ensure this is less than 65536
        bestEffort=True
    ).reproject(crs='EPSG:4326', scale=30)

# Apply spatial pooling to each image in the yearly collection
pooled_collection = yearly_ts.map(spatial_pool)

# Download the pooled image collection directly as xarray using wxee
# Convert the pooled collection to xarray
# Ensure to specify the region and scale
ds = pooled_collection.wx.to_xarray(region=aoi, scale=30)

# Fill NaN values and clip to appropriate ranges
ds = ds.fillna(-1)
ds = ds.clip(min=-128, max=127)
ds["label"] = ds["label"].astype("int8")
ds["x"] = ds["x"].astype("float32")
ds["y"] = ds["y"].astype("float32")
ds["time"] = ds["time"].dt.strftime("%Y-%m")

# Save the pooled dataset back to NetCDF
ds.to_netcdf(r"Data/DW/2020_parana_delta_mode_pooled_30m.nc")


Requesting data:   0%|          | 0/1 [00:00<?, ?it/s]

In [10]:
# Chaco-Santiago
aoi = border_area

# Load the Dynamic World dataset and filter by AOI and date range
dw_filtered = wxee.TimeSeries("GOOGLE/DYNAMICWORLD/V1") \
                .filterDate("2020-01-01", "2021-01-01") \
                .filterBounds(aoi) \
                .select("label")

# Clip images to the area of interest
def clip_image(image):
    return image.clip(aoi)

# Apply the clipping function to each image in the collection
dw_filtered = dw_filtered.map(clip_image)

# Aggregate temporally to get the mode for each year
yearly_ts = dw_filtered.aggregate_time("year", ee.Reducer.mode())

# Function to spatially pool the data using a 30m resolution
def spatial_pool(image):
    # Set the default projection for the image
    image = image.setDefaultProjection('EPSG:4326', None, 30)  # Set the projection with a scale of 30m

    return image.reduceResolution(
        reducer=ee.Reducer.mode(),
        maxPixels=65535,  # Ensure this is less than 65536
        bestEffort=True
    ).reproject(crs='EPSG:4326', scale=30)

# Apply spatial pooling to each image in the yearly collection
pooled_collection = yearly_ts.map(spatial_pool)

# Download the pooled image collection directly as xarray using wxee
# Convert the pooled collection to xarray
# Ensure to specify the region and scale
ds = pooled_collection.wx.to_xarray(region=aoi, scale=30)

# Fill NaN values and clip to appropriate ranges
ds = ds.fillna(-1)
ds = ds.clip(min=-128, max=127)
ds["label"] = ds["label"].astype("int8")
ds["x"] = ds["x"].astype("float32")
ds["y"] = ds["y"].astype("float32")
ds["time"] = ds["time"].dt.strftime("%Y-%m")

# Save the pooled dataset back to NetCDF
ds.to_netcdf(r"Data/DW/2020_border_area_mode_pooled_30m.nc")

Requesting data:   0%|          | 0/1 [00:00<?, ?it/s]

## Agregación espacial y temporal de todas las bandas con media aritmética y se vuelve a calcular la clase predominante


In [5]:
# Paraná
aoi = parana_delta

CLASS_NAMES = ['water', 'trees', 'grass', 'flooded_vegetation', 'crops', 'shrub_and_scrub', 'built', 'bare', 'snow_and_ice']
CLASS_NUMBER = ee.List.sequence(0, 8)

# Load the Dynamic World dataset and filter by AOI and date range
dw_filtered = wxee.TimeSeries("GOOGLE/DYNAMICWORLD/V1") \
    .filterDate("2020-01-01", "2021-01-01") \
    .filterBounds(aoi)

# Clip images to the area of interest
def clip_image(image):
    return image.clip(aoi)

# Apply the clipping function to each image in the collection
dw_filtered = dw_filtered.map(clip_image)

# Aggregate temporally to get the mode for each year
yearly_ts = dw_filtered.aggregate_time("year", ee.Reducer.mean())

# Function to spatially pool the data using a 30m resolution
def spatial_pool(image):
    # Set the default projection for the image
    image = image.setDefaultProjection('EPSG:4326', None, 30)  # Set the projection with a scale of 30m

    return image.reduceResolution(
        reducer=ee.Reducer.mean(),
        maxPixels=65535,  # Ensure this is less than 65536
        bestEffort=True
    ).reproject(crs='EPSG:4326', scale=30)

# Apply spatial pooling to each image in the yearly collection
pooled_collection = yearly_ts.map(spatial_pool)


yearly_dominant_class = pooled_collection.map(
    lambda image:
    image.select(CLASS_NAMES)
    .reduce(ee.Reducer.max())
    .eq(image.select(CLASS_NAMES))
    .multiply(ee.Image.constant(CLASS_NUMBER))
    .reduce(ee.Reducer.sum())
    .rename('label')
    .uint8()
    .copyProperties(image, ['system:time_start'])
)


# Download the pooled image collection directly as xarray using wxee
# Convert the pooled collection to xarray
# Ensure to specify the region and scale
ds = yearly_dominant_class.wx.to_xarray(region=aoi, scale=30)

# Fill NaN values and clip to appropriate ranges
ds = ds.fillna(-1)
ds = ds.clip(min=-128, max=127)
ds["label"] = ds["label"].astype("int8")
ds["x"] = ds["x"].astype("float32")
ds["y"] = ds["y"].astype("float32")
ds["time"] = ds["time"].dt.strftime("%Y-%m")

# Save the pooled dataset back to NetCDF
ds.to_netcdf(r"Data/DW/2020_parana_delta_mean_pooled_30m.nc")

Requesting data:   0%|          | 0/1 [00:00<?, ?it/s]

In [11]:
# Descarga Chaco-Santiago

aoi = border_area

CLASS_NAMES = ['water', 'trees', 'grass', 'flooded_vegetation', 'crops', 'shrub_and_scrub', 'built', 'bare', 'snow_and_ice']
CLASS_NUMBER = ee.List.sequence(0, 8)

# Load the Dynamic World dataset and filter by AOI and date range
dw_filtered = wxee.TimeSeries("GOOGLE/DYNAMICWORLD/V1") \
    .filterDate("2020-01-01", "2021-01-01") \
    .filterBounds(aoi) \

# Clip images to the area of interest
def clip_image(image):
    return image.clip(aoi)

# Apply the clipping function to each image in the collection
dw_filtered = dw_filtered.map(clip_image)

# Aggregate temporally to get the mode for each year
yearly_ts = dw_filtered.aggregate_time("year", ee.Reducer.mean())

# Function to spatially pool the data using a 30m resolution
def spatial_pool(image):
    # Set the default projection for the image
    image = image.setDefaultProjection('EPSG:4326', None, 30)  # Set the projection with a scale of 30m

    return image.reduceResolution(
        reducer=ee.Reducer.mean(),
        maxPixels=65535,  # Ensure this is less than 65536
        bestEffort=True
    ).reproject(crs='EPSG:4326', scale=30)

# Apply spatial pooling to each image in the yearly collection
pooled_collection = yearly_ts.map(spatial_pool)


yearly_dominant_class = pooled_collection.map(
    lambda image:
    image.select(CLASS_NAMES)
    .reduce(ee.Reducer.max())
    .eq(image.select(CLASS_NAMES))
    .multiply(ee.Image.constant(CLASS_NUMBER))
    .reduce(ee.Reducer.sum())
    .rename('label')
    .uint8()
    .copyProperties(image, ['system:time_start'])
)


# Download the pooled image collection directly as xarray using wxee
# Convert the pooled collection to xarray
# Ensure to specify the region and scale
ds = yearly_dominant_class.wx.to_xarray(region=aoi, scale=30)

# Fill NaN values and clip to appropriate ranges
ds = ds.fillna(-1)
ds = ds.clip(min=-128, max=127)
ds["label"] = ds["label"].astype("int8")
ds["x"] = ds["x"].astype("float32")
ds["y"] = ds["y"].astype("float32")
ds["time"] = ds["time"].dt.strftime("%Y-%m")

# Save the pooled dataset back to NetCDF
ds.to_netcdf(r"Data/DW/2020_border_area_mean_pooled_30m.nc")

Requesting data:   0%|          | 0/1 [00:00<?, ?it/s]

## Agregación temporal de todas las bandas con media aritmética y se vuelve a calcular la clase predominante. Espacialmente no se hace pooling


In [13]:
# Paraná
aoi = parana_delta

CLASS_NAMES = ['water', 'trees', 'grass', 'flooded_vegetation', 'crops', 'shrub_and_scrub', 'built', 'bare', 'snow_and_ice']
CLASS_NUMBER = ee.List.sequence(0, 8)

# Load the Dynamic World dataset and filter by AOI and date range
dw_filtered = wxee.TimeSeries("GOOGLE/DYNAMICWORLD/V1") \
    .filterDate("2020-01-01", "2021-01-01") \
    .filterBounds(aoi)

# Clip images to the area of interest
def clip_image(image):
    return image.clip(aoi)

# Apply the clipping function to each image in the collection
dw_filtered = dw_filtered.map(clip_image)

# Aggregate temporally to get the mode for each year
yearly_ts = dw_filtered.aggregate_time("year", ee.Reducer.mean())

yearly_dominant_class = yearly_ts.map(
    lambda image:
    image.select(CLASS_NAMES)
    .reduce(ee.Reducer.max())
    .eq(image.select(CLASS_NAMES))
    .multiply(ee.Image.constant(CLASS_NUMBER))
    .reduce(ee.Reducer.sum())
    .rename('label')
    .uint8()
    .copyProperties(image, ['system:time_start'])
)


# Download the pooled image collection directly as xarray using wxee
# Convert the pooled collection to xarray
# Ensure to specify the region and scale
ds = yearly_dominant_class.wx.to_xarray(region=aoi, scale=30)

# Fill NaN values and clip to appropriate ranges
ds = ds.fillna(-1)
ds = ds.clip(min=-128, max=127)
ds["label"] = ds["label"].astype("int8")
ds["x"] = ds["x"].astype("float32")
ds["y"] = ds["y"].astype("float32")
ds["time"] = ds["time"].dt.strftime("%Y-%m")

# Save the pooled dataset back to NetCDF
ds.to_netcdf(r"Data/DW/2020_parana_delta_mean_unpooled_30m.nc")

Requesting data:   0%|          | 0/1 [00:00<?, ?it/s]

In [12]:
# Descarga Chaco-Santiago


aoi = border_area

CLASS_NAMES = ['water', 'trees', 'grass', 'flooded_vegetation', 'crops', 'shrub_and_scrub', 'built', 'bare', 'snow_and_ice']
CLASS_NUMBER = ee.List.sequence(0, 8)

# Load the Dynamic World dataset and filter by AOI and date range
dw_filtered = wxee.TimeSeries("GOOGLE/DYNAMICWORLD/V1") \
    .filterDate("2020-01-01", "2021-01-01") \
    .filterBounds(aoi)

# Clip images to the area of interest
def clip_image(image):
    return image.clip(aoi)

# Apply the clipping function to each image in the collection
dw_filtered = dw_filtered.map(clip_image)

# Aggregate temporally to get the mode for each year
yearly_ts = dw_filtered.aggregate_time("year", ee.Reducer.mean())

yearly_dominant_class = yearly_ts.map(
    lambda image:
    image.select(CLASS_NAMES)
    .reduce(ee.Reducer.max())
    .eq(image.select(CLASS_NAMES))
    .multiply(ee.Image.constant(CLASS_NUMBER))
    .reduce(ee.Reducer.sum())
    .rename('label')
    .uint8()
    .copyProperties(image, ['system:time_start'])
)


# Download the pooled image collection directly as xarray using wxee
# Convert the pooled collection to xarray
# Ensure to specify the region and scale
ds = yearly_dominant_class.wx.to_xarray(region=aoi, scale=30)

# Fill NaN values and clip to appropriate ranges
ds = ds.fillna(-1)
ds = ds.clip(min=-128, max=127)
ds["label"] = ds["label"].astype("int8")
ds["x"] = ds["x"].astype("float32")
ds["y"] = ds["y"].astype("float32")
ds["time"] = ds["time"].dt.strftime("%Y-%m")

# Save the pooled dataset back to NetCDF
ds.to_netcdf(r"Data/DW/2020_border_area_mean_unpooled_30m.nc")

Requesting data:   0%|          | 0/1 [00:00<?, ?it/s]